## LLM

In [6]:
import sys
sys.path.insert(0, '../src/')

import matplotlib.pyplot as plt
from tokenizer import TokenizerBPE
from data_handling import normalize_to_ascii, clean_text
from utils import saver, loader
import numpy as np
import re

import os
import pickle as pkl
from tqdm.notebook import tqdm
import json

# disable gpu for testing purposes
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


## Hotpot

In [7]:
with open('../corpus/hotpot_train_v1.1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [8]:
length = len(data)

In [9]:
context_list = []
question_list = []
answer_list = []


for i, sample in tqdm(enumerate(data), total=length):
    
    supporting_facts = sample["supporting_facts"]
    supporting_facts = set([fact[0] for fact in supporting_facts])
    context = sample["context"]
    context_accum = []
    for fact in supporting_facts:
        for paragraph in context:
            if fact == paragraph[0]:
                context_accum.extend(paragraph[1])

    context = " ".join(context_accum)
    context_list.append(clean_text(normalize_to_ascii(context)))

    q = clean_text(normalize_to_ascii(sample["question"]))
    a = clean_text(normalize_to_ascii(sample["answer"]))
    question_list.append(q)
    answer_list.append(a)

  0%|          | 0/90447 [00:00<?, ?it/s]

In [10]:
saver('../corpus/hotpot_raw', [context_list, question_list, answer_list])

## No Redundancy

In [11]:
context_list = []
question_list = []
answer_list = []


for i, sample in tqdm(enumerate(data), total=length):
    
    supporting_facts = sample["supporting_facts"]
    context = sample["context"]
    context_accum = []
    for fact, idx in supporting_facts:
        for paragraph in context:
            if fact == paragraph[0]:
                try:
                    context_accum.append(clean_text(normalize_to_ascii(paragraph[1][idx])))
                except IndexError:
                    print(f"IndexError for fact {fact} at index {idx} in paragraph {paragraph[0]}")
                    continue
        
    context_list.append(" ".join(context_accum))
                                    
    q = clean_text(normalize_to_ascii(sample["question"]))
    a = clean_text(normalize_to_ascii(sample["answer"]))
    question_list.append(q)
    answer_list.append(a)

  0%|          | 0/90447 [00:00<?, ?it/s]

IndexError for fact Minoru Suzuki at index 2 in paragraph Minoru Suzuki
IndexError for fact Jonathan Taylor Thomas at index 20 in paragraph Jonathan Taylor Thomas
IndexError for fact Francis Poulenc at index 3 in paragraph Francis Poulenc
IndexError for fact Pan's Labyrinth at index 2 in paragraph Pan's Labyrinth
IndexError for fact Rupert Grint at index 4 in paragraph Rupert Grint
IndexError for fact Walkerville, South Australia at index 52 in paragraph Walkerville, South Australia
IndexError for fact Khady Sylla at index 2 in paragraph Khady Sylla
IndexError for fact Robbie Rist at index 52 in paragraph Robbie Rist
IndexError for fact Harry Styles at index 4 in paragraph Harry Styles
IndexError for fact William T. Anderson at index 2 in paragraph William T. Anderson
IndexError for fact South Park (season 12) at index 20 in paragraph South Park (season 12)
IndexError for fact Hutterite at index 40 in paragraph Hutterite
IndexError for fact 2013 Moore tornado at index 2 in paragraph 20

In [12]:
saver('../corpus/hotpot_noRed_raw', [context_list, question_list, answer_list])

## 